In [8]:
import pandas as pd
import numpy as np

# Utilise directement le nom de fichier que vous semblez avoir
df = pd.read_csv('data/global_df_test.csv' , parse_dates=['Date'])

# Affiche les infos essentielles
df.info()
print(f"\nDimensions: {df.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91637 entries, 0 to 91636
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   MovieID     91637 non-null  int64         
 1   CustomerID  91637 non-null  int64         
 2   rating      91637 non-null  int64         
 3   Date        91637 non-null  datetime64[ns]
 4   Year        91637 non-null  int64         
 5   Name        91637 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 4.2+ MB

Dimensions: (91637, 6)


# Mapper les IDs

In [9]:

# 1. Mapper CustomerID -> user_idx
unique_users = df['CustomerID'].unique()
n_users = len(unique_users)
user_to_idx = {original_id: new_idx for new_idx, original_id in enumerate(unique_users)}
df['user_idx'] = df['CustomerID'].map(user_to_idx)



In [10]:
# 2. Mapper MovieID -> movie_idx
unique_movies = df['MovieID'].unique()
n_movies = len(unique_movies)
movie_to_idx = {original_id: new_idx for new_idx, original_id in enumerate(unique_movies)}
df['movie_idx'] = df['MovieID'].map(movie_to_idx)

In [13]:

print(f"  Utilisateurs uniques trouvés (n_users) : {n_users}")
print(f"  Films uniques trouvés (n_movies)      : {n_movies}")


  Utilisateurs uniques trouvés (n_users) : 2000
  Films uniques trouvés (n_movies)      : 50


In [14]:
print(df[['CustomerID', 'user_idx', 'MovieID', 'movie_idx']].head())

   CustomerID  user_idx  MovieID  movie_idx
0     1468812         0     2372          0
1      306466         1     2372          0
2     1907667         2     2372          0
3     1488844         3     2372          0
4      923325         4     2372          0


In [16]:
df.head()

,MovieID,CustomerID,rating,Date,Year,Name,user_idx,movie_idx
0,2372,1468812,5,2004-12-10,2004,The Bourne Supremacy,0,0
1,2372,306466,4,2004-12-14,2004,The Bourne Supremacy,1,0
2,2372,1907667,4,2004-12-30,2004,The Bourne Supremacy,2,0
3,2372,1488844,5,2005-04-04,2004,The Bourne Supremacy,3,0
4,2372,923325,4,2005-03-22,2004,The Bourne Supremacy,4,0


# Trier les Données
**Mettre toutes les lignes (notes) de chaque utilisateur ensemble, et pour chaque utilisateur, ordonner ses notes de la plus ancienne à la plus récente.**

In [21]:

df.sort_values(by=['user_idx', 'Date'], ascending=True, inplace=True)

# réinitialiser l'index
# drop=True évite de garder l'ancien index comme une nouvelle colonne
df.reset_index(drop=True, inplace=True)


df.head(10)

,MovieID,CustomerID,rating,Date,Year,Name,user_idx,movie_idx
0,2862,1468812,5,2001-02-19,1991,The Silence of the Lambs,0,11
1,2953,1468812,5,2001-02-19,1987,Moonstruck,0,14
2,3605,1468812,4,2001-02-19,1939,The Wizard of Oz: Collector's Edition,0,25
3,4227,1468812,4,2001-02-19,1997,The Full Monty,0,41
4,4330,1468812,3,2001-02-19,1995,While You Were Sleeping,0,44
5,4306,1468812,5,2001-02-21,1999,The Sixth Sense,0,43
6,4262,1468812,4,2001-02-27,1999,Sleepy Hollow,0,42
7,3610,1468812,4,2001-03-14,1992,Lethal Weapon 3,0,26
8,3433,1468812,3,2001-04-04,1992,Forever Young,0,24
9,3782,1468812,4,2001-04-04,1990,Flatliners,0,30


# groupe par utilisateur
**Obtenir, pour chaque user_idx, la liste ordonnée des movie_idx qu'il a notés et la liste ordonnée des Rating correspondants.**

In [25]:
df.groupby('user_idx')['movie_idx'].apply(list)

user_idx
0       [11, 14, 25, 41, 44, 43, 42, 26, 24, 30, 15, 9...
1       [43, 42, 14, 25, 2, 47, 11, 26, 41, 24, 44, 9,...
2       [2, 11, 15, 39, 41, 43, 47, 19, 8, 9, 17, 23, ...
3       [7, 23, 36, 40, 0, 35, 48, 4, 16, 10, 17, 24, ...
4       [16, 21, 34, 0, 2, 3, 4, 7, 8, 9, 10, 20, 22, ...
                              ...                        
1995    [2, 3, 4, 5, 7, 14, 15, 22, 23, 24, 25, 28, 29...
1996    [7, 14, 25, 28, 29, 30, 44, 2, 4, 8, 9, 10, 11...
1997    [2, 5, 11, 15, 19, 25, 24, 43, 9, 41, 6, 39, 4...
1998    [4, 5, 6, 7, 9, 10, 14, 17, 22, 23, 25, 27, 29...
1999    [14, 2, 5, 25, 40, 41, 43, 47, 9, 11, 24, 26, ...
Name: movie_idx, Length: 2000, dtype: object

In [29]:

# Appliquer 'list' à la colonne 'movie_idx' pour chaque groupe d'user_idx
user_movie_sequences = df.groupby('user_idx')['movie_idx'].apply(list)

In [27]:

# Faire de même pour la colonne 'Rating'
user_rating_sequences = df.groupby('user_idx')['rating'].apply(list)

In [31]:
print(f"Nombre de séquences utilisateur extraites: {len(user_movie_sequences)} ") # Doit être égal à n_users


Nombre de séquences utilisateur extraites: 2000 


**Créer des paires (séquence_entrée -> note_cible) où chaque séquence_entrée a une longueur fixe (max_len) et est potentiellement complétée par du "padding" (remplissage) si l'historique est trop court.**

In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [34]:
max_len = 20          # Longueur fixe souhaitée
pad_value_movie = n_movies # Valeur pour padder les films
pad_value_rating = 0       # Valeur pour padder les notes


In [35]:
# Listes pour stocker les séquences AVANT padding
unpadded_movie_seqs = []
unpadded_rating_seqs = []
all_target_ratings = []

# Boucle pour extraire les sous-séquences et cibles
for user_idx in user_movie_sequences.index:
    movie_seq = user_movie_sequences[user_idx]
    rating_seq = user_rating_sequences[user_idx]
    for i in range(1, len(movie_seq)):
        start_index = max(0, i - max_len)
        # On ajoute les sous-séquences (non paddées) aux listes
        unpadded_movie_seqs.append(movie_seq[start_index:i])
        unpadded_rating_seqs.append(rating_seq[start_index:i])
        # On ajoute la cible
        all_target_ratings.append(rating_seq[i])

In [ ]:

total_samples = len(all_target_ratings)
print(f"Nombre d'échantillons bruts: {len(all_target_ratings)}")

# --- Padding et Conversion NumPy (après la boucle) ---
if total_samples > 0:
    print("Padding des séquences et conversion NumPy...")
    # Utiliser pad_sequences de Keras
    # 'padding='pre'' ajoute le padding au début
    # 'truncating='pre'' coupe le début si la séquence est trop longue (ne devrait pas arriver ici)
    X_movie_seqs_np = pad_sequences(unpadded_movie_seqs, maxlen=max_len,
                                    padding='pre', truncating='pre',
                                    value=pad_value_movie) # Précise la valeur de padding

    X_rating_seqs_np = pad_sequences(unpadded_rating_seqs, maxlen=max_len,
                                     padding='pre', truncating='pre',
                                     value=pad_value_rating) # Précise la valeur de padding

    # Convertir les cibles
    y_target_ratings_np = np.array(all_target_ratings, dtype=np.uint8)

    # Ajuster les types après padding (pad_sequences renvoie souvent int32)
    movie_dtype = np.uint16 if pad_value_movie < 65535 else np.uint32
    X_movie_seqs_np = X_movie_seqs_np.astype(movie_dtype)
    X_rating_seqs_np = X_rating_seqs_np.astype(np.uint8)

    print("Padding et conversion terminés.")

    # --- Vérification ---
    print("\nFormes des tableaux NumPy créés:")
    print(f"  X Movies : {X_movie_seqs_np.shape}")
    print(f"  X Ratings: {X_rating_seqs_np.shape}")
    print(f"  y Target : {y_target_ratings_np.shape}")
    print("\nExemple (1er échantillon paddé):")
    print(f"  Input Movies : {X_movie_seqs_np[0]}")
    print(f"  Input Ratings: {X_rating_seqs_np[0]}")
    print(f"  Target Rating: {y_target_ratings_np[0]}")

else:
    print("Aucun échantillon généré.")
    X_movie_seqs_np, X_rating_seqs_np, y_target_ratings_np = np.array([]), np.array([]), np.array([])

Nombre d'échantillons bruts: 89637
Padding des séquences et conversion NumPy...
Padding et conversion terminés.

Formes des tableaux NumPy créés:
  X Movies : (89637, 20)
  X Ratings: (89637, 20)
  y Target : (89637,)

Exemple (1er échantillon paddé):
  Input Movies : [50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 11]
  Input Ratings: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5]
  Target Rating: 5


# Diviser les Données (Train/Validation)

In [38]:
from sklearn.model_selection import train_test_split


# Paramètres
validation_size = 0.2 # Taille de l'ensemble de validation (20%)
random_seed = 42    # Pour obtenir la même division à chaque fois

# Diviser les indices (car on a plusieurs X à diviser de la même façon)
num_samples = X_movie_seqs_np.shape[0]
indices = np.arange(num_samples)
train_indices, val_indices = train_test_split(
    indices,
    test_size=validation_size,
    random_state=random_seed
)

# Créer les ensembles finaux en utilisant les indices
X_train_movies = X_movie_seqs_np[train_indices]
X_train_ratings = X_rating_seqs_np[train_indices]
y_train = y_target_ratings_np[train_indices]

X_val_movies = X_movie_seqs_np[val_indices]
X_val_ratings = X_rating_seqs_np[val_indices]
y_val = y_target_ratings_np[val_indices]


# --- Vérification Rapide ---
print("\nFormes des ensembles après division:")
print(f"  Train - Movies: {X_train_movies.shape}, Ratings: {X_train_ratings.shape}, Target: {y_train.shape}")
print(f"  Val   - Movies: {X_val_movies.shape}, Ratings: {X_val_ratings.shape}, Target: {y_val.shape}")


Formes des ensembles après division:
  Train - Movies: (71709, 20), Ratings: (71709, 20), Target: (71709,)
  Val   - Movies: (17928, 20), Ratings: (17928, 20), Target: (17928,)


# Définir l'Architecture du Modèle RNN